In [1]:
import pymysql
import pandas as pd
doris_host='101.6.5.211'
doris_port =9030
doris_user='root'
doris_password=''
doris_db='tpch_sf100'


In [8]:
conn=pymysql.connect(host=doris_host,port=doris_port, user=doris_user, passwd=doris_password, db=doris_db)
# query="explain optimized plan select c_nationkey, count(*) from customer group by c_nationkey;"
# query="show column stats lineitem(l_tax);"
query = "explain optimized plan select c_custkey, o_custkey from customer join orders on c_custkey = o_custkey where c_custkey < 10;"

In [10]:
q1 = """
select
    l_returnflag,
    l_linestatus,
    sum(l_quantity) as sum_qty,
    sum(l_extendedprice) as sum_base_price,
    sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
    sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
    avg(l_quantity) as avg_qty,
    avg(l_extendedprice) as avg_price,
    avg(l_discount) as avg_disc,
    count(*) as count_order
from
    lineitem
where
    l_shipdate <= date '1998-12-01' - interval '90' day
group by
    l_returnflag,
    l_linestatus
order by
    l_returnflag,
    l_linestatus;
"""
query = "explain optimized plan" + query

In [12]:
q2 = """
select
    s_acctbal,
    s_name,
    n_name,
    p_partkey,
    p_mfgr,
    s_address,
    s_phone,
    s_comment
from
    part,
    supplier,
    partsupp,
    nation,
    region
where
    p_partkey = ps_partkey
    and s_suppkey = ps_suppkey
    and p_size = 15
    and p_type like '%BRASS'
    and s_nationkey = n_nationkey
    and n_regionkey = r_regionkey
    and r_name = 'EUROPE'
    and ps_supplycost = (
        select
            min(ps_supplycost)
        from
            partsupp,
            supplier,
            nation,
            region
        where
        p_partkey = ps_partkey
        and s_suppkey = ps_suppkey
        and s_nationkey = n_nationkey
        and n_regionkey = r_regionkey
        and r_name = 'EUROPE'
)
order by
    s_acctbal desc,
    n_name,
    s_name,
    p_partkey
"""
query = "explain optimized plan" + q2

In [13]:
df=pd.read_sql_query(query, conn)
print(df)

/tmp/ipykernel_2960448/3660904836.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query, conn)


                      Explain String(Nereids Planner)
0                         cost = 1.8395694842674473E8
1   PhysicalResultSink[3628] ( outputExprs=[s_acct...
2   +--PhysicalQuickSort[3625]@22 ( orderKeys=[s_a...
3      +--PhysicalDistribute[3622]@24 ( distributi...
4         +--PhysicalQuickSort[3619]@24 ( orderKey...
5            +--PhysicalProject[3616]@21 ( project...
6               +--PhysicalFilter[3613]@20 ( predi...
7                  +--PhysicalWindow[3610]@19 ( wi...
8                     +--PhysicalQuickSort[3607]@1...
9                        +--PhysicalDistribute[360...
10                          +--PhysicalProject[360...
11                             +--PhysicalHashJoin...
12                                |--PhysicalDistr...
13                                |  +--PhysicalPr...
14                                |     +--Physica...
15                                |        |--Phys...
16                                |        |  +--P...
17                          

In [14]:
for i in range(len(df)):
    print(df.iloc[i].item())

cost = 1.8395694842674473E8
PhysicalResultSink[3628] ( outputExprs=[s_acctbal#14, s_name#10, n_name#22, p_partkey#0, p_mfgr#2, s_address#11, s_phone#13, s_comment#15] )
+--PhysicalQuickSort[3625]@22 ( orderKeys=[s_acctbal#14 desc, n_name#22 asc null first, s_name#10 asc null first, p_partkey#0 asc null first], phase=MERGE_SORT, stats=1 1 )
   +--PhysicalDistribute[3622]@24 ( distributionSpec=DistributionSpecGather, stats=1 1 )
      +--PhysicalQuickSort[3619]@24 ( orderKeys=[s_acctbal#14 desc, n_name#22 asc null first, s_name#10 asc null first, p_partkey#0 asc null first], phase=LOCAL_SORT, stats=1 1 )
         +--PhysicalProject[3616]@21 ( projects=[s_acctbal#14, s_name#10, n_name#22, p_partkey#0, p_mfgr#2, s_address#11, s_phone#13, s_comment#15], stats=1 1 )
            +--PhysicalFilter[3613]@20 ( predicates=(ps_supplycost#19 = min(ps_supplycost) OVER(PARTITION BY p_partkey)#48), stats=1 1 )
               +--PhysicalWindow[3610]@19 ( windowFrameGroup=(Funcs=[min(ps_supplycost#19) W

In [11]:
cursor=conn.cursor()
cursor.execute(query)
rows = cursor.fetchall()
print(rows)
plan=""
for row in rows:
    plan+='\n' + row[0]
print(plan)

(('cost = 4.500005325E7',), ('PhysicalResultSink[138] ( outputExprs=[c_nationkey#3, __count_1#8] )',), ('+--PhysicalDistribute[135]@2 ( distributionSpec=DistributionSpecGather, stats=25 1 )',), ('   +--PhysicalHashAggregate[132]@2 ( aggPhase=GLOBAL, aggMode=BUFFER_TO_RESULT, maybeUseStreaming=false, groupByExpr=[c_nationkey#3], outputExpr=[c_nationkey#3, count(partial_count(*)#9) AS `count(*)`#8], partitionExpr=Optional[[c_nationkey#3]], requireProperties=[DistributionSpecHash ( orderedShuffledColumns=[3], shuffleType=REQUIRE, tableId=-1, selectedIndexId=-1, partitionIds=[], equivalenceExprIds=[[3]], exprIdToEquivalenceSet={3=0} ) Order: ([])], stats=25 1 )',), ('      +--PhysicalDistribute[129]@4 ( distributionSpec=DistributionSpecHash ( orderedShuffledColumns=[3], shuffleType=EXECUTION_BUCKETED, tableId=-1, selectedIndexId=-1, partitionIds=[], equivalenceExprIds=[[3]], exprIdToEquivalenceSet={3=0} ), stats=25 1 )',), ('         +--PhysicalHashAggregate[126]@4 ( aggPhase=LOCAL, aggMod